In [ ]:
Seven Days to Spark with Scala : Vivek Mangipudi

In [ ]:
#Simple car data set exploration: Day 1

In [ ]:
Loading data in memory
Creating SQLContext
Creating Spark DataFrame
Group data by columns
Operating on columns
Running SQL Queries from a Spark DataFrame

In [5]:

import sys.process._
import java.net.URL
import java.io.File

def fileDownloader(url: String, filename: String) = {
    new URL(url) #> new File(filename) !!
}

fileDownloader("https://gist.githubusercontent.com/seankross/a412dfbd88b3db70b74b/raw/5f23f993cd87c283ce766e7ac6b329ee7cc2e1d1/mtcars.csv", "/resources/mtcars.csv")

""

In [6]:
import au.com.bytecode.opencsv.CSVParser
val sqlContext = new org.apache.spark.sql.SQLContext(sc)
import sqlContext.implicits._

In [7]:
// Define the schema using a case class.
case class Cars(car: String, mpg: String, cyl: String, disp: String, hp: String, drat: String, wt: String, qsec: String, vs: String, am: String, gear: String, carb: String)

In [9]:
val csv = sc.textFile("/resources/data/mtcars1.csv")
val headerAndRows = csv.map(line => line.split(",").map(_.trim))
val header = headerAndRows.first
val data = headerAndRows.filter(_(0) != header(0))
val mtcars = data.map(p => Cars(p(0), p(1), p(2), p(3), p(4), p(5), p(6), p(7), p(8), p(9), p(10), p(11))).toDF()
mtcars.printSchema

root
 |-- car: string (nullable = true)
 |-- mpg: string (nullable = true)
 |-- cyl: string (nullable = true)
 |-- disp: string (nullable = true)
 |-- hp: string (nullable = true)
 |-- drat: string (nullable = true)
 |-- wt: string (nullable = true)
 |-- qsec: string (nullable = true)
 |-- vs: string (nullable = true)
 |-- am: string (nullable = true)
 |-- gear: string (nullable = true)
 |-- carb: string (nullable = true)



In [10]:
mtcars.show(5)

+-----------------+----+---+----+---+----+-----+-----+---+---+----+----+
|              car| mpg|cyl|disp| hp|drat|   wt| qsec| vs| am|gear|carb|
+-----------------+----+---+----+---+----+-----+-----+---+---+----+----+
|        Mazda RX4|  21|  6| 160|110| 3.9| 2.62|16.46|  0|  1|   4|   4|
|    Mazda RX4 Wag|  21|  6| 160|110| 3.9|2.875|17.02|  0|  1|   4|   4|
|       Datsun 710|22.8|  4| 108| 93|3.85| 2.32|18.61|  1|  1|   4|   1|
|   Hornet 4 Drive|21.4|  6| 258|110|3.08|3.215|19.44|  1|  0|   3|   1|
|Hornet Sportabout|18.7|  8| 360|175|3.15| 3.44|17.02|  0|  0|   3|   2|
+-----------------+----+---+----+---+----+-----+-----+---+---+----+----+
only showing top 5 rows



In [11]:
mtcars.select("mpg").show(5)

+----+
| mpg|
+----+
|  21|
|  21|
|22.8|
|21.4|
|18.7|
+----+
only showing top 5 rows



In [14]:
mtcars.select("car","mpg","hp").show(5)

+-----------------+----+---+
|              car| mpg| hp|
+-----------------+----+---+
|        Mazda RX4|  21|110|
|    Mazda RX4 Wag|  21|110|
|       Datsun 710|22.8| 93|
|   Hornet 4 Drive|21.4|110|
|Hornet Sportabout|18.7|175|
+-----------------+----+---+
only showing top 5 rows



In [15]:
mtcars.filter(mtcars("mpg") < 18).show(5)

+-----------+----+---+-----+---+----+----+-----+---+---+----+----+
|        car| mpg|cyl| disp| hp|drat|  wt| qsec| vs| am|gear|carb|
+-----------+----+---+-----+---+----+----+-----+---+---+----+----+
| Duster 360|14.3|  8|  360|245|3.21|3.57|15.84|  0|  0|   3|   4|
|  Merc 280C|17.8|  6|167.6|123|3.92|3.44| 18.9|  1|  0|   4|   4|
| Merc 450SE|16.4|  8|275.8|180|3.07|4.07| 17.4|  0|  0|   3|   3|
| Merc 450SL|17.3|  8|275.8|180|3.07|3.73| 17.6|  0|  0|   3|   3|
|Merc 450SLC|15.2|  8|275.8|180|3.07|3.78|   18|  0|  0|   3|   3|
+-----------+----+---+-----+---+----+----+-----+---+---+----+----+
only showing top 5 rows



In [16]:
mtcars.withColumn("wtTon", mtcars("wt") * 0.45).show(6)

+-----------------+----+---+----+---+----+-----+-----+---+---+----+----+-------+
|              car| mpg|cyl|disp| hp|drat|   wt| qsec| vs| am|gear|carb|  wtTon|
+-----------------+----+---+----+---+----+-----+-----+---+---+----+----+-------+
|        Mazda RX4|  21|  6| 160|110| 3.9| 2.62|16.46|  0|  1|   4|   4|  1.179|
|    Mazda RX4 Wag|  21|  6| 160|110| 3.9|2.875|17.02|  0|  1|   4|   4|1.29375|
|       Datsun 710|22.8|  4| 108| 93|3.85| 2.32|18.61|  1|  1|   4|   1|  1.044|
|   Hornet 4 Drive|21.4|  6| 258|110|3.08|3.215|19.44|  1|  0|   3|   1|1.44675|
|Hornet Sportabout|18.7|  8| 360|175|3.15| 3.44|17.02|  0|  0|   3|   2|  1.548|
|          Valiant|18.1|  6| 225|105|2.76| 3.46|20.22|  1|  0|   3|   1|  1.557|
+-----------------+----+---+----+---+----+-----+-----+---+---+----+----+-------+
only showing top 6 rows



In [17]:
import org.apache.spark.sql.functions._
mtcars.groupBy("cyl").agg(avg("wt")).show(5)

+---+-----------------+
|cyl|          avg(wt)|
+---+-----------------+
|  4|2.285727272727273|
|  6|3.117142857142857|
|  8|3.999214285714286|
+---+-----------------+



In [20]:
mtcars.groupBy("gear").agg(avg("wt")).show(5)

+----+------------------+
|gear|           avg(wt)|
+----+------------------+
|   3|            3.8926|
|   4|2.6166666666666667|
|   5|            2.6326|
+----+------------------+



In [21]:
mtcars.groupBy("cyl").agg(count("wt")).sort($"count(wt)".desc).show(5)

+---+---------+
|cyl|count(wt)|
+---+---------+
|  8|       14|
|  4|       11|
|  6|        7|
+---+---------+



In [22]:
//Register this DataFrame as a table.
mtcars.registerTempTable("cars")

// SQL statements can be run by using the sql methods provided by sqlContext.

val highgearcars = sqlContext.sql("SELECT gear FROM cars WHERE cyl >= 4 AND cyl <= 9")
highgearcars.show(6)

+----+
|gear|
+----+
|   4|
|   4|
|   4|
|   3|
|   3|
|   3|
+----+
only showing top 6 rows

